In [13]:
import os
import h5py

# import matplotlib.pyplot as plt
import time, pickle, pandas

import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2D, Input
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras import backend
from keras import optimizers
from keras import layers

In [11]:
# import ssl
# from sklearn.datasets import fetch_lfw_people
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# import numpy as np

# ssl._create_default_https_context = ssl._create_unverified_context


# class LFWLoader:
#     """
#     Params:
#         data_home: path to store LFW data
#         target_names (ndarray): list of target names
#     """

#     def __init__(self, data_home='~/Downloads/scikit_learn_data'):
#         """
#         Args:
#              data_home (str, optional): Default to '~/Downloads/scikit_learn_data'
#         """
#         self.data_home = data_home

#     def load_lfw_data(self,
#                       funneled=True,
#                       resize=1,
#                       min_faces_per_person=2,
#                       test_size=0.25,
#                       random_state=42):
#         """
#         Args:
#             funneled (boolean, optional): Default to True. Download the funneled dataset.
#             resize (float, optional): Default to 1. Ratio to resize the face pictures.
#             min_faces_per_person (int, optional): Default to 2. The extracted dataset will only retain pictures of people that have at least min_faces_per_person different pictures.
#             test_size (float, int, optional): Default to 0.25. If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. If int, represents the absolute number of test samples.
#             random_state (int, RandomState instance, optional): Default to 42. If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator.
#         Returns:
#             X_train (ndarray)
#             X_test (ndarray)
#             y_train (ndarray): onehot label
#             y_test (ndarray): onehot label
#         """
#         # #############################################################################
#         # Download the data, if not already on disk and load it as numpy arrays

#         lfw_people = fetch_lfw_people(
#             data_home=self.data_home,
#             funneled=funneled,
#             resize=resize,
#             min_faces_per_person=min_faces_per_person,
#             color=True
#             )

#         # introspect the images arrays to find the shapes (for plotting)
#         n_samples, d, h, w = lfw_people.images.shape

#         # for machine learning we use the 2 data directly (as relative pixel
#         # positions info is ignored by this model)
#         X = lfw_people.images
#         n_features = X.shape[1]

#         # the label to predict is the id of the person
#         y = lfw_people.target
#         y = OneHotEncoder(sparse=False).fit_transform(np.reshape(y, (len(y), 1)))

#         self.target_names = lfw_people.target_names # store the list of target names
#         n_classes = self.target_names.shape[0]

#         print("Total dataset size:")
#         print("n_samples: %d" % n_samples)
#         print("n_features: %d" % n_features)
#         print("n_classes: %d" % n_classes)
#         # print("image size: " + str(h) + ", " + str(w))

#         # #############################################################################
#         # Split into a training set and a test set using a stratified k fold

#         # split into a training and testing set
#         X_train, X_test, y_train, y_test = train_test_split(
#             X, y, test_size=test_size, random_state=random_state)

#         print('Done!')
#         return X_train, X_test, y_train, y_test

In [12]:
batch_size = 16
nb_train = 14513
nb_test = 1791
img_width, img_height = 250, 250
train_data_dir = '../CACD2000/train'
validation_data_dir = '../CACD2000/val'


train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=nb_train)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=nb_test)

Found 14513 images belonging to 200 classes.
Found 1791 images belonging to 200 classes.


In [14]:
tensorboard_callback = TensorBoard(log_dir='./logs/cacd2000', histogram_freq=0, write_graph=True, write_images=False)
checkpoint = ModelCheckpoint("./models/cacd2000.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [14]:
# # from keras.models import Model

# eva_data_dir = './data/c_test'

# eva_datagen = ImageDataGenerator(rescale=1./255)

# eva_generator = eva_datagen.flow_from_directory(
#         eva_data_dir,
#         target_size=(img_width, img_height),
#         batch_size=batch_size)

# tf_model.evaluate_generator(eva_generator)

### Resnet

In [15]:
def res_block(input_tensor, kernel_size, filters, strides=(1, 1)):
    """
    Construct a residual block
                        
                        |------------------------>
    1 x 1 conv, filter1      |
                        |                        |
    kernel_size x kernel_size conv, filter2      |
                        |<-----------------------                         |
    
    Args:
        input_tensor (Tensor): input data
        kernel_size (int): size of 2nd layers of conv2d
        filters (tuple of int): 1 * 3 filter value of 1st conv and 2nd conv
    
    Reference: 
        https://blog.waya.ai/deep-residual-learning-9610bb62c355
        https://blog.csdn.net/wspba/article/details/56019373
        https://blog.csdn.net/qq_25491201/article/details/78405549
    """
#     shortcut = input_tensor
    filter1, filter2 = filters
#     x = Conv2D(filter1, strides = strides, kernel_size=(1, 1), padding='same')(input_tensor)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
    
    x = Conv2D(filter1, kernel_size=kernel_size, strides=strides, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filter2, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
#     x = Conv2D(filter3, kernel_size=(1, 1), padding='same')(x)
#     x = BatchNormalization()(x)
    
    shortcut = Conv2D(filter2, kernel_size=(1, 1), strides=strides, padding='same')(input_tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    
    return x
    
              

In [16]:
def build_resnet(framework='tf', nb_res = (3, 0, 0, 0),input_shape=(3, 250, 250), nc=200):
    """
    """

    if framework == 'th':
        # build the VGG16 network in Theano weight ordering mode
        backend.set_image_dim_ordering('th')
    else:
        # build the VGG16 network in Tensorflow weight ordering mode
        backend.set_image_dim_ordering('tf')
    
    input_tensor = Input(shape=input_shape)
    x = input_tensor

    # residual units
    
    for i in range(nb_res[0]):
        x = res_block(x, 3, (64, 64))
    
    for i in range(nb_res[1]):
        x = res_block(x, 3, (128, 128))
        
    for i in range(nb_res[2]):
        x = res_block(x, 3, (256, 256))
       
    for i in range(nb_res[3]):
        x = res_block(x, 3, (512, 512))
    
    x = layers.GlobalAveragePooling2D()(x)
    
    x = Dense(units=nc, kernel_initializer="he_normal",
                      activation="softmax")(x)
    
    model = Model(input_tensor, x)
    return model

In [17]:
# X_train, X_test, y_train, y_test = LFWLoader().load_lfw_data()
X_train, y_train = train_generator.next()
X_test, y_test = test_generator.next()

In [18]:
print('Total dataset size')
print('train data size = ' + str(X_train.shape))
print('test data size = ' + str(y_test.shape))


Total dataset size
train data size = (14513, 250, 250, 3)
test data size = (1791, 200)


In [19]:
# input_shape = X_train.shape[1:]
# model = build_resnet(input_shape=input_shape, nc=200) # tensorflow dim ordering is like this
# model.summary()

In [20]:
# model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
#               metrics=['accuracy'])

In [21]:
# model.fit(x=X_train, 
#           y=y_train, 
#           batch_size=1, 
#           epochs=10, 
#           verbose=1, 
#           validation_data = (X_test, y_test),
#           callbacks=[tensorboard_callback, checkpoint],
#           shuffle=True)

## VGG + ResNet

In [17]:
def build_vgg16(framework='tf'):

    if framework == 'th':
        # build the VGG16 network in Theano weight ordering mode
        backend.set_image_dim_ordering('th')
    else:
        # build the VGG16 network in Tensorflow weight ordering mode
        backend.set_image_dim_ordering('tf')
        
    model = Sequential()
    if framework == 'th':
        model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))
    else:
        model.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height, 3)))
        
    model.add(Conv2D(64, (3, 3), activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='conv5_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    return model

build_vgg16().summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_27 (ZeroPaddi (None, 252, 252, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 250, 250, 64)      1792      
_________________________________________________________________
zero_padding2d_28 (ZeroPaddi (None, 252, 252, 64)      0         
_________________________________________________________________
conv1_2 (Conv2D)             (None, 250, 250, 64)      36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 125, 125, 64)      0         
_________________________________________________________________
zero_padding2d_29 (ZeroPaddi (None, 127, 127, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 125, 125, 128)     73856     
__________

In [18]:
# path to the model weights files.
weights_path = '/home/zhijian_jiang95/imagenet_vgg16_fine-tuning/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
img_width = 250
img_height = 250
tf_model = build_vgg16('tf')
# tf_model.load_weights(weights_path)

In [19]:
# build a classifier model to put on top of the convolutional model
tf_model.add(build_resnet(input_shape=(7, 7, 512)))

In [20]:
tf_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_40 (ZeroPaddi (None, 252, 252, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 250, 250, 64)      1792      
_________________________________________________________________
zero_padding2d_41 (ZeroPaddi (None, 252, 252, 64)      0         
_________________________________________________________________
conv1_2 (Conv2D)             (None, 250, 250, 64)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 125, 125, 64)      0         
_________________________________________________________________
zero_padding2d_42 (ZeroPaddi (None, 127, 127, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 125, 125, 128)     73856     
__________

In [26]:
# # set the first 25 layers (up to the last conv block)
# # to non-trainable (weights will not be updated)
# for layer in tf_model.layers[:25]:
#     layer.trainable = False

# # compile the model with a SGD/momentum optimizer
# # and a very slow learning rate.
# tf_model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
#               metrics=['accuracy'])

In [27]:
# tf_model.load_weights('./models/cacd2000.h5')

In [28]:
# tf_model.fit(x=X_train, 
#           y=y_train, 
#           batch_size=16, 
#           epochs=40, 
#           verbose=1, 
#           validation_data = (X_test, y_test),
#           callbacks=[tensorboard_callback, checkpoint],
#           shuffle=True)

In [29]:
# # change learning rate from 1e-4 to 1e-5
# tf_model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
#               metrics=['accuracy'])

In [30]:
# tf_model.fit(x=X_train, 
#           y=y_train, 
#           batch_size=16, 
#           epochs=20, 
#           verbose=1, 
#           validation_data = (X_test, y_test),
#           callbacks=[tensorboard_callback, checkpoint],
#           shuffle=True)

In [8]:
tf_model.load_weights('./models/cacd2000.h5')

ValueError: You are trying to load a weight file containing 14 layers into a model with 13 layers.

In [32]:
# # 1e-5 -> 1e-6
# tf_model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.SGD(lr=1e-6, momentum=0.9),
#               metrics=['accuracy'])

In [33]:
# tf_model.fit(x=X_train, 
#           y=y_train, 
#           batch_size=16, 
#           epochs=20, 
#           verbose=1, 
#           validation_data = (X_test, y_test),
#           callbacks=[tensorboard_callback, checkpoint],
#           shuffle=True)

In [34]:
# # 1e-6 -> 1e-7
# tf_model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.SGD(lr=1e-7, momentum=0.9),
#               metrics=['accuracy'])

In [35]:
# tf_model.fit(x=X_train, 
#           y=y_train, 
#           batch_size=16, 
#           epochs=10, 
#           verbose=1, 
#           validation_data = (X_test, y_test),
#           callbacks=[tensorboard_callback, checkpoint],
#           shuffle=True)

In [36]:
# # sgd -> adam
# tf_model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.Adam(lr=1e-7, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# ,
#               metrics=['accuracy'])

In [37]:
# tf_model.fit(x=X_train, 
#           y=y_train, 
#           batch_size=16, 
#           epochs=10, 
#           verbose=1, 
#           validation_data = (X_test, y_test),
#           callbacks=[tensorboard_callback, checkpoint],
#           shuffle=True)

In [43]:
# unfreeze VGG part
for layer in tf_model.layers[:25]:
    layer.trainable = True

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
tf_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
              metrics=['accuracy'])

In [44]:
tf_model.load_weights('./models/cacd2000.h5')
import math
lr=1e-5
past_loss = 2.60467 # min loss before unfreeze
while lr > 1e-12:
    print('Current learning rate = ' + str(lr))
    tf_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=0.9),
              metrics=['accuracy'])
    tf_model.fit(x=X_train, 
              y=y_train, 
              batch_size=16, 
              epochs=1, 
              verbose=1, 
              validation_data = (X_test, y_test),
              callbacks=[tensorboard_callback, checkpoint],
              shuffle=True)
    loss, acc = tf_model.evaluate(X_test, y_test, verbose=0)
    print('past loss = ' + str(past_loss) + ' loss = ' + str(loss))
    if past_loss < loss:
        lr = lr * 0.1 # update lr if loss not improve
    else:
        past_loss = loss # update loss if loss improve

Current learning rate = 1e-05
Train on 14513 samples, validate on 1791 samples
Epoch 1/1
14513/14513 [==============================] - 676s 47ms/step - loss: 1.9190 - acc: 0.6604 - val_loss: 2.4341 - val_acc: 0.5070

Epoch 00001: val_loss did not improve
Current learning rate = 1e-05
Train on 14513 samples, validate on 1791 samples
Epoch 1/1
14513/14513 [==============================] - 678s 47ms/step - loss: 1.8514 - acc: 0.6808 - val_loss: 2.4254 - val_acc: 0.4964

Epoch 00001: val_loss did not improve
Current learning rate = 1e-05
Train on 14513 samples, validate on 1791 samples
Epoch 1/1
14513/14513 [==============================] - 678s 47ms/step - loss: 1.7938 - acc: 0.6982 - val_loss: 2.3850 - val_acc: 0.5109

Epoch 00001: val_loss improved from 2.41463 to 2.38502, saving model to ./models/cacd2000.h5
Current learning rate = 1e-05
Train on 14513 samples, validate on 1791 samples
Epoch 1/1
14513/14513 [==============================] - 679s 47ms/step - loss: 1.7524 - acc: 0.71

### Data Augmentation

In [21]:
batch_size = 16
nb_train = 14513
nb_test = 1791
img_width, img_height = 250, 250
train_data_dir = '../CACD2000/train'
validation_data_dir = '../CACD2000/val'


train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=nb_train)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=nb_test)

Found 14513 images belonging to 200 classes.
Found 1791 images belonging to 200 classes.


In [22]:
# X_train, X_test, y_train, y_test = LFWLoader().load_lfw_data()
X_train, y_train = train_generator.next()
X_test, y_test = test_generator.next()

In [23]:
tf_model.load_weights('./models/cacd2000.h5')
import math
lr=1e-5
past_loss = 2.34500 # min loss before unfreeze
while lr > 1e-12:
    print('Current learning rate = ' + str(lr))
    tf_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=0.9),
              metrics=['accuracy'])
    tf_model.fit(x=X_train, 
              y=y_train, 
              batch_size=16, 
              epochs=1, 
              verbose=1, 
              validation_data = (X_test, y_test),
              callbacks=[tensorboard_callback, checkpoint],
              shuffle=True)
    loss, acc = tf_model.evaluate(X_test, y_test, verbose=0)
    print('past loss = ' + str(past_loss) + ' loss = ' + str(loss))
    if past_loss < loss:
        lr = lr * 0.1 # update lr if loss not improve
    else:
        past_loss = loss # update loss if loss improve

Current learning rate = 1e-05
Train on 14513 samples, validate on 1791 samples
Epoch 1/1
14513/14513 [==============================] - 703s 48ms/step - loss: 2.2036 - acc: 0.5722 - val_loss: 2.3304 - val_acc: 0.5248

Epoch 00001: val_loss improved from inf to 2.33042, saving model to ./models/cacd2000.h5
past loss = 2.345 loss = 2.3304226375837556
Current learning rate = 1e-05
Train on 14513 samples, validate on 1791 samples
Epoch 1/1
14513/14513 [==============================] - 681s 47ms/step - loss: 2.0937 - acc: 0.6089 - val_loss: 2.2871 - val_acc: 0.5293

Epoch 00001: val_loss improved from 2.33042 to 2.28707, saving model to ./models/cacd2000.h5
past loss = 2.3304226375837556 loss = 2.2870698890361223
Current learning rate = 1e-05
Train on 14513 samples, validate on 1791 samples
Epoch 1/1
14513/14513 [==============================] - 681s 47ms/step - loss: 2.0048 - acc: 0.6358 - val_loss: 2.3259 - val_acc: 0.5159

Epoch 00001: val_loss did not improve
past loss = 2.28706988903